In [5]:
# !pip install scipy
# !pip install sklearn
!pip install tensorflow==2.5

ERROR: Could not find a version that satisfies the requirement tensorflow==2.5 (from versions: none)
ERROR: No matching distribution found for tensorflow==2.5


In [6]:
import numpy as np
import cv2
import os
import pickle
import sys
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Implementing Bag of Features Model

def orb_features(images):
  orb_descriptors_class_by_class={}
  orb_descriptors_list=[]
  orb=cv2.xfeatures2d.ORB_create()
  for key,value in images.items():
    print(key, "Started")
    features=[]
    for img in value:
      kp,desc=orb.detectAndCompute(img,None)
      orb_descriptors_list.extend(desc)
      features.append(desc)
    orb_descriptors_class_by_class[key]=features
    print(key," Completed!")
  return [orb_descriptors_list,orb_descriptors_class_by_class]

In [ ]:

# Creating a visual dictionary using only the train dataset 
# K-means clustering alogo takes only 2 parameters which are number of clusters (k) and descrpitors list
# It reurn an array which holds central points

def minibatchkmeans(k, descriptors_list):
  kmeans=MiniBatchKMeans(n_clusters=k)
  print("MiniBatchKMeans Initialized!")
  kmeans.fit(descriptors_list)
  print("Clusters Created!")
  visual_words=kmeans.cluster_centers_
  return visual_words, kmeans


In [ ]:
# Loading train images into dictionaries which holds all images category by category

def load_images_by_category(folder):
  images={}
  for label in os.listdir(folder):
    print(label," started")
    category=[]
    path=folder+'/'+label
    for image in os.listdir(path):
      img=cv2.imread(path+'/'+image)
      #new_img=cv2.resize(img,(128,128))
      if img is not None:
        category.append(img)
    images[label]=category
    print(label, "ended")
  return images

In [ ]:

# Creating histograms for train images

# Function takes 2 parameters. The first one is a dictionary that holds the descriptors that are separated class by class 
# And the second parameter is the clustered model
# Returns a dictionary that holds the histograms for each images that are separated class by class. 

def create_histogram(all_descs,kmeans):
  features_dict={}
  for key,value in all_descs.items():
    print(key," Started!")
    category=[]
    for desc in value:
      raw_words=kmeans.predict(desc)
      hist = np.array(np.bincount(raw_words,minlength=n_classes*clustering_factor))
      category.append(hist)
    features_dict[key]=category
    print(key," Completed!")
  return features_dict

In [ ]:
train_folder='C:/Users/Sandeep Roy/Desktop/data2/data/Train-Test/Train'

# Load train images
train_images=load_images_by_category(train_folder)
#print(len(train_images))

#print(len(train_images['a'][0][0]))

#Extracting orb features from each image stored in train_images list

orbs=orb_features(train_images)
all_train_descriptors=orbs[0]
train_descriptors_by_class=orbs[1]

#print(len(orbs[0]))
#print(len(orbs[1]['0'][1]))

# Calling MiniBatchkmeans function and getting central points
visual_words,kmeans=minibatchkmeans(n_classes*clustering_factor,all_train_descriptors)


# Calling create_histogram and getting histogram for each image
bows_train=create_histogram(train_descriptors_by_class,kmeans)

#print((bows_train['a'][0][1]))

# Saving .csv file
import csv
loc='cnn files/train.csv'
with open(loc,'w',newline='') as file:
  writer=csv.writer(file)
  header=[]
  for i in range (1,n_classes*clustering_factor):
    header.append(str('pixel')+str(i))
  header.append('Label')
  writer.writerow(header)
  count=0
  for label in bows_train:
     # print(len(bows_train[label]))
    for i in range(len(bows_train[label])):
      list=[]
      for j in range(150):
        list.append(bows_train[label][i][j])
      list.append(label)
      writer.writerow(list)